# ***Creating Neural Network***

In [3]:
import torch
import torch.nn as nn # For nural netwok
import torch.optim as optim #For optimization oparation
from sklearn.datasets import fetch_california_housing #for take some data
from sklearn.preprocessing import StandardScaler #for normalization
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader #Dataloder => every package to train
from tqdm import tqdm #For progress bar

In [10]:
#inharit the data set and make my own dataset
class HousingDataset(Dataset):
  def __init__(self,data,targets):
    self.data = data
    self.targets = targets

  def __len__(self):
    return len(self.targets)

  def __getitem__(self, idx):# will take the id and show us the objects
        x = torch.tensor(self.data[idx], dtype=torch.float32)
        y = torch.tensor(self.targets[idx], dtype=torch.float32)
        return x, y

In [5]:
data = fetch_california_housing()
x = data['data']
y = data['target']

In [22]:
# Perform data preprocessing
scaler = StandardScaler()
x = scaler.fit_transform(x)
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.3,random_state= 42)

train_dataset = HousingDataset(x_train,y_train)
test_dataset = HousingDataset(x_test,y_test)
train_dataloder = DataLoader(train_dataset,batch_size =64,shuffle = True)
test_dataloder = DataLoader(test_dataset,batch_size= 64)

In [23]:
# Define a neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        # Initialize the parent class (nn.Module)
        super(NeuralNetwork, self).__init__()

        # First fully connected (dense) layer
        # input_size  -> number of features in input data
        # 64          -> number of neurons in hidden layer
        self.fc1 = nn.Linear(input_size, 64)

        # ReLU activation function (adds non-linearity)
        self.relu = nn.ReLU()

        # Output layer
        # 64 -> hidden neurons
        # 1  -> single output value (for regression)
        self.fc2 = nn.Linear(64, 1)

    # Forward pass: defines how data flows through the network
    def forward(self, x):
        x = self.fc1(x)   # Linear transformation
        x = self.relu(x)  # Apply activation function
        x = self.fc2(x)   # Final output layer
        return x


In [24]:
# Create model
model = NeuralNetwork(input_size=x_train.shape[1])
# Loss function (Mean Squared Error for regression)
criterion = nn.MSELoss()
# Optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Device configuration (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Convert training data to tensors
x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
x_test = torch.tensor(x_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)


In [26]:
# Display a sample from the dataset
print("Sample from the dataset:")

sample_idx = 0  # index of the sample to inspect
sample_input, sample_target = train_dataset[sample_idx]

print("Input:", sample_input)     # feature vector
print("Target:", sample_target)   # corresponding label
print()

# Training configuration
num_epochs = 10  # number of full passes over the training dataset
# Training and evaluation loop
for epoch in range(num_epochs):

    # Training phase
    model.train()            # set model to training mode
    train_loss = 0.0         # accumulator for training loss

    # Iterate over training batches
    for inputs, targets in tqdm(train_dataloder):

        # Move batch data to CPU/GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass: model prediction
        outputs = model(inputs)

        # Compute loss (squeeze removes extra dimension: [B,1] -> [B])
        loss = criterion(outputs.squeeze(), targets)

        # Backward pass
        optimizer.zero_grad()  # clear previous gradients
        loss.backward()        # compute gradients
        optimizer.step()       # update model parameters

        # Accumulate loss
        train_loss += loss.item()

    # Average training loss for this epoch
    train_loss /= len(train_dataloder)

    #Evaluation phase
    model.eval()              # set model to evaluation mode
    test_loss = 0.0           # accumulator for test loss

    # Disable gradient computation during evaluation
    with torch.no_grad():
        for inputs, targets in test_dataloder:

            # Move test batch to CPU/GPU
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs.squeeze(), targets)

            # Accumulate test loss
            test_loss += loss.item()

    # Average test loss for this epoch
    test_loss /= len(test_dataloder)
    # Logging results
    print(
        f"Epoch [{epoch + 1}/{num_epochs}] | "
        f"Train Loss: {train_loss:.4f} | "
        f"Test Loss: {test_loss:.4f}"
    )


Sample from the dataset:
Input: tensor([ 0.1371,  0.5054,  0.1832, -0.2557, -0.1832, -0.0082, -0.7968,  0.7735])
Target: tensor(1.9380)



100%|██████████| 226/226 [00:00<00:00, 413.87it/s]


Epoch [1/10] | Train Loss: 0.3752 | Test Loss: 0.3729


100%|██████████| 226/226 [00:00<00:00, 405.07it/s]


Epoch [2/10] | Train Loss: 0.3703 | Test Loss: 0.3684


...

Epoch [8/10] | Train Loss: 0.3484 | Test Loss: 0.3507


100%|██████████| 226/226 [00:00<00:00, 319.94it/s]


Epoch [9/10] | Train Loss: 0.3463 | Test Loss: 0.3490


100%|██████████| 226/226 [00:00<00:00, 330.60it/s]


Epoch [10/10] | Train Loss: 0.3440 | Test Loss: 0.3482
